In [5]:
import os

# home_dir = '/content/gdrive/My Drive/AChina' 
# home_dir = '/hy-tmp'
# os.chdir(home_dir)
# pwd

!pip install tqdm 
!pip install fire

from glob import glob
from tqdm import tqdm

import numpy as np
import pandas as pd

import os
import warnings
import sys

import fire

# warnings.filterwarnings("ignore")  # avoid printing out absolute paths

# os.chdir("../../..")

!pip install scipy
!pip install torch pytorch-lightning pytorch_forecasting

import copy
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAPE, SMAPE, PoissonLoss, QuantileLoss
from matplotlib import pyplot as plt
from pytorch_lightning.utilities.model_summary import summarize
from pytorch_forecasting import TemporalFusionTransformer
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from matplotlib import pyplot as plt

from multiprocessing import Pool, freeze_support

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.
Looking in indexes: https://mirrors.aliyun.com/pypi/simple
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.
Looking in indexes: https://mirrors.aliyun.com/pypi/simple
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.
Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 826.4/826.4 KB 273.1 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.4/141.4 KB 252.7 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 701.2/701.2 KB 245.9 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 KB 252.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 239.3 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
scrop = 'rice'
alidata = pd.read_csv(f'../ALIDATASET_{scrop}.csv')

display(alidata)

,Unnamed: 0,time_idx,county,year,month,MOD_month_t,b0b0,b0b1,b0b2,b0b3,...,Wind_f_tavg_254,rice_sownarea,avg_rice_sownarea,med_rice_sownarea,rice_yieldval,avg_rice_yieldval,med_rice_yieldval,rice_yield,avg_rice_yield,med_rice_yield
0,0,0,0,2003,1,0,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.9,0.45,0.22,1.8012,0.84,0.74
1,1,1,0,2003,1,1,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.9,0.45,0.22,1.8012,0.84,0.74
2,2,2,0,2003,1,2,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.9,0.45,0.22,1.8012,0.84,0.74
3,3,3,0,2003,1,3,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.9,0.45,0.22,1.8012,0.84,0.74
4,4,4,0,2003,2,0,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.9,0.45,0.22,1.8012,0.84,0.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24160,24160,40,30,2022,11,3,0.0,0.0,0.0,1.505150,...,0.0,9999.90,75.13,74.54,9999999.9,62.83,65.31,1000.0100,0.83,0.87
24161,24161,41,30,2022,12,0,0.0,0.0,0.0,0.845098,...,0.0,9999.90,75.13,74.54,9999999.9,62.83,65.31,1000.0100,0.83,0.87
24162,24162,42,30,2022,12,1,0.0,0.0,0.0,1.602060,...,0.0,9999.90,75.13,74.54,9999999.9,62.83,65.31,1000.0100,0.83,0.87
24163,24163,43,30,2022,12,2,0.0,0.0,0.0,1.301030,...,0.0,9999.90,75.13,74.54,9999999.9,62.83,65.31,1000.0100,0.83,0.87


In [21]:
infer_mask = alidata['year'].isin([2019, 2020, 2021, 2022])

data_infer = alidata[infer_mask]

data_infer['rice_sownarea'] = np.nan
data_infer['rice_yieldval'] = np.nan
data_infer['rice_yield']    = np.nan

years = [x for x in range(2003, 2019)]

test_year = 2018

years.remove(test_year)

train_mask = alidata['year'].isin(years)
data = alidata[train_mask]

test_mask = alidata['year'].isin([test_year])
data_test = alidata[test_mask]

display(data_infer)

display(data)

display(data_test)


/tmp/ipykernel_5137/952164515.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_infer['rice_sownarea'] = np.nan
/tmp/ipykernel_5137/952164515.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_infer['rice_yieldval'] = np.nan
/tmp/ipykernel_5137/952164515.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

,Unnamed: 0,time_idx,county,year,month,MOD_month_t,b0b0,b0b1,b0b2,b0b3,...,Wind_f_tavg_254,rice_sownarea,avg_rice_sownarea,med_rice_sownarea,rice_yieldval,avg_rice_yieldval,med_rice_yieldval,rice_yield,avg_rice_yield,med_rice_yield
716,19332,0,0,2019,1,0,0.0,0.0,0.0,0.000000,...,0.0,NaN,0.44,0.26,NaN,0.45,0.22,NaN,0.84,0.74
717,19333,1,0,2019,1,1,0.0,0.0,0.0,0.000000,...,0.0,NaN,0.44,0.26,NaN,0.45,0.22,NaN,0.84,0.74
718,19334,2,0,2019,1,2,0.0,0.0,0.0,0.000000,...,0.0,NaN,0.44,0.26,NaN,0.45,0.22,NaN,0.84,0.74
719,19335,3,0,2019,1,3,0.0,0.0,0.0,0.000000,...,0.0,NaN,0.44,0.26,NaN,0.45,0.22,NaN,0.84,0.74
720,19336,4,0,2019,2,0,0.0,0.0,0.0,0.000000,...,0.0,NaN,0.44,0.26,NaN,0.45,0.22,NaN,0.84,0.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24160,24160,40,30,2022,11,3,0.0,0.0,0.0,1.505150,...,0.0,NaN,75.13,74.54,NaN,62.83,65.31,NaN,0.83,0.87
24161,24161,41,30,2022,12,0,0.0,0.0,0.0,0.845098,...,0.0,NaN,75.13,74.54,NaN,62.83,65.31,NaN,0.83,0.87
24162,24162,42,30,2022,12,1,0.0,0.0,0.0,1.602060,...,0.0,NaN,75.13,74.54,NaN,62.83,65.31,NaN,0.83,0.87
24163,24163,43,30,2022,12,2,0.0,0.0,0.0,1.301030,...,0.0,NaN,75.13,74.54,NaN,62.83,65.31,NaN,0.83,0.87


,Unnamed: 0,time_idx,county,year,month,MOD_month_t,b0b0,b0b1,b0b2,b0b3,...,Wind_f_tavg_254,rice_sownarea,avg_rice_sownarea,med_rice_sownarea,rice_yieldval,avg_rice_yieldval,med_rice_yieldval,rice_yield,avg_rice_yield,med_rice_yield
0,0,0,0,2003,1,0,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.90,0.45,0.22,1.8012,0.84,0.74
1,1,1,0,2003,1,1,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.90,0.45,0.22,1.8012,0.84,0.74
2,2,2,0,2003,1,2,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.90,0.45,0.22,1.8012,0.84,0.74
3,3,3,0,2003,1,3,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.90,0.45,0.22,1.8012,0.84,0.74
4,4,4,0,2003,2,0,0.0,0.0,0.0,0.000000,...,0.0,1.61,0.44,0.26,2.90,0.45,0.22,1.8012,0.84,0.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23936,18112,40,30,2017,11,3,0.0,0.0,0.0,1.322219,...,0.0,74.24,75.13,74.54,65.47,62.83,65.31,0.8819,0.83,0.87
23937,18113,41,30,2017,12,0,0.0,0.0,0.0,1.851258,...,0.0,74.24,75.13,74.54,65.47,62.83,65.31,0.8819,0.83,0.87
23938,18114,42,30,2017,12,1,0.0,0.0,0.0,1.982271,...,0.0,74.24,75.13,74.54,65.47,62.83,65.31,0.8819,0.83,0.87
23939,18115,43,30,2017,12,2,0.0,0.0,0.0,2.212188,...,0.0,74.24,75.13,74.54,65.47,62.83,65.31,0.8819,0.83,0.87


,Unnamed: 0,time_idx,county,year,month,MOD_month_t,b0b0,b0b1,b0b2,b0b3,...,Wind_f_tavg_254,rice_sownarea,avg_rice_sownarea,med_rice_sownarea,rice_yieldval,avg_rice_yieldval,med_rice_yieldval,rice_yield,avg_rice_yield,med_rice_yield
671,18117,0,0,2018,1,0,0.0,0.0,0.0,0.000000,...,0.0,0.17,0.44,0.26,0.07,0.45,0.22,0.4118,0.84,0.74
672,18118,1,0,2018,1,1,0.0,0.0,0.0,0.000000,...,0.0,0.17,0.44,0.26,0.07,0.45,0.22,0.4118,0.84,0.74
673,18119,2,0,2018,1,2,0.0,0.0,0.0,0.000000,...,0.0,0.17,0.44,0.26,0.07,0.45,0.22,0.4118,0.84,0.74
674,18120,3,0,2018,1,3,0.0,0.0,0.0,0.000000,...,0.0,0.17,0.44,0.26,0.07,0.45,0.22,0.4118,0.84,0.74
675,18121,4,0,2018,2,0,0.0,0.0,0.0,0.000000,...,0.0,0.17,0.44,0.26,0.07,0.45,0.22,0.4118,0.84,0.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23981,19327,40,30,2018,11,3,0.0,0.0,0.0,1.672098,...,0.0,78.39,75.13,74.54,72.65,62.83,65.31,0.9268,0.83,0.87
23982,19328,41,30,2018,12,0,0.0,0.0,0.0,1.397940,...,0.0,78.39,75.13,74.54,72.65,62.83,65.31,0.9268,0.83,0.87
23983,19329,42,30,2018,12,1,0.0,0.0,0.0,1.397940,...,0.0,78.39,75.13,74.54,72.65,62.83,65.31,0.9268,0.83,0.87
23984,19330,43,30,2018,12,2,0.0,0.0,0.0,2.301030,...,0.0,78.39,75.13,74.54,72.65,62.83,65.31,0.9268,0.83,0.87


In [25]:
max_prediction_length = 16
max_encoder_length = 27
training_cutoff = data["time_idx"].max() - max_prediction_length

avg_med = ["avg_rice_yield", "med_rice_yield", "avg_rice_sownarea", "med_rice_sownarea",\
                 "avg_rice_yieldval", "med_rice_yieldval"]

_static_reals = avg_med

mod_names = [f'b{iband}b{bins}' for iband in range(9) for bins in range(512)]

famine_list = ['Evap_tavg', 'LWdown_f_tavg', 'Lwnet_tavg', 'Psurf_f_tavg', 'Qair_f_tavg', 'Qg_tavg', 'Qh_tavg', 'Qle_tavg',\
            'Qs_tavg', 'Qsb_tavg', 'RadT_tavg', 'Rainf_f_tavg', 'SnowCover_inst', 'SnowDepth_inst', 'Snowf_tavg', \
            'SoilMoi00_10cm_tavg', 'SoilMoi10_40cm_tavg', 'SoilMoi40_100cm_tavg', \
            'SoilTemp00_10cm_tavg', 'SoilTemp10_40cm_tavg', 'SoilTemp40_100cm_tavg', \
            'SWdown_f_tavg', 'SWE_inst', 'Swnet_tavg', 'Tair_f_tavg', 'Wind_f_tavg']

nbins = ['_' + str(x) for x in range(0, 255)]

famine_names = [famine + bb for famine in famine_list for bb in nbins]

# display( data[ [ col for col in mod_names ] ] )

# display( data[ [ col for col in famine_names ] ] )

# fn

# for iband in range(len(famine_list)):
#     ibands_name = [famine_list[iband] + x for x in nbins]
# print(famine_list[iband])

_time_varying_unknown_reals = []

training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="rice_yield",
    group_ids=["county", "year"],
    min_encoder_length=max_encoder_length // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,
    min_prediction_length = 1,
    max_prediction_length = max_prediction_length,
    static_categoricals = ["agency", "sku"],
    static_reals = _static_reals,
    time_varying_known_categoricals=["special_days", "month"],
    # variable_groups={"special_days": special_days},  # group of categorical variables can be treated as one variable
    time_varying_known_reals=["time_idx", "price_regular", "discount_in_percent"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[
        "volume",
        "log_volume",
        "industry_volume",
        "soda_volume",
        "avg_max_temp",
        "avg_volume_by_agency",
        "avg_volume_by_sku",
    ],
    target_normalizer=GroupNormalizer(
        groups=["agency", "sku"], transformation="softplus"
    ),  # use softplus and normalize by group
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

# create validation set (predict=True) which means to predict the last max_prediction_length points in time
# for each series
validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)

# create dataloaders for model
batch_size = 128  # set this between 32 to 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

,b0b0,b0b1,b0b2,b0b3,b0b4,b0b5,b0b6,b0b7,b0b8,b0b9,...,b8b502,b8b503,b8b504,b8b505,b8b506,b8b507,b8b508,b8b509,b8b510,b8b511
0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23936,0.0,0.0,0.0,1.322219,1.255273,1.698970,1.568202,1.897627,1.785330,1.886491,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23937,0.0,0.0,0.0,1.851258,0.778151,0.698970,0.602060,1.301030,1.505150,1.531479,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23938,0.0,0.0,0.0,1.982271,1.079181,1.176091,1.278754,1.792392,1.740363,1.812913,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23939,0.0,0.0,0.0,2.212188,1.579784,1.612784,1.732394,1.812913,1.819544,1.954243,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,Evap_tavg_0,Evap_tavg_1,Evap_tavg_2,Evap_tavg_3,Evap_tavg_4,Evap_tavg_5,Evap_tavg_6,Evap_tavg_7,Evap_tavg_8,Evap_tavg_9,...,Wind_f_tavg_245,Wind_f_tavg_246,Wind_f_tavg_247,Wind_f_tavg_248,Wind_f_tavg_249,Wind_f_tavg_250,Wind_f_tavg_251,Wind_f_tavg_252,Wind_f_tavg_253,Wind_f_tavg_254
0,0.0,0.0,0.0,0.0,0.000000,0.000000,1.863323,3.823213,3.852053,3.686100,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.000000,1.863323,3.823213,3.852053,3.686100,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.000000,1.863323,3.823213,3.852053,3.686100,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.000000,1.863323,3.823213,3.852053,3.686100,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,1.491362,3.558709,4.127396,3.258877,1.919078,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23936,0.0,0.0,0.0,0.0,0.000000,4.692697,4.918219,4.164710,3.810233,3.890868,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23937,0.0,0.0,0.0,0.0,3.023664,5.057423,4.430607,4.133986,3.811039,4.036150,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23938,0.0,0.0,0.0,0.0,3.023664,5.057423,4.430607,4.133986,3.811039,4.036150,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23939,0.0,0.0,0.0,0.0,3.023664,5.057423,4.430607,4.133986,3.811039,4.036150,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


NameError: name 'fn' is not defined